<a href="https://colab.research.google.com/github/vksriharsha/Aminobert_ESM2_Comparative_Study/blob/main/Aminobert_ESM2_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries


In [1]:
!pip install torch
!pip install fair-esm
!pip install fair-esm[esmfold]
!pip install Bio
!pip install biopython
# OpenFold and its remaining dependency
!pip install 'dllogger @ git+https://github.com/NVIDIA/dllogger.git'
!pip install 'openfold @ git+https://github.com/aqlaboratory/openfold.git@4b41059694619831a7db195b7e0988fc4ff3a307'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 12.9 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 510 kB 42.5 MB/s 
     |████████████████████████████████| 41 kB 564 kB/s 
     |████████████████████████████████| 3.1 MB 53.4 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 145 kB 72.4 MB/s 
     |████████████████████████████████| 15.2 MB 57.1 MB/s 
     |████████████████████████████████| 117 kB 73.6 MB/s 
     |████████████████████████████████| 125 kB 75.2 MB/s 
     |██████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 272 kB 20.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/dllogger.git to /tmp/pip-install-u9h3ml4y/dllogger_7b7eb60995724238a2e435af78cc19dc
  Running command git clone -q https://github.com/NVIDIA/dllogger.git /tmp/pip-install-u9h3ml4y/dllogger_7b7eb60995724238a2e435af78cc19dc
  Created wheel for dllogger: filename=DLLogger-1.0.0-py3-none-any.whl size=5670 sha256=aa5423db6d5032eb78d4f60927e0c1acca73ea170110c465815e47c81d14c01b
  Stored in directory: /tmp/pip-ephem-wheel-cache-9fnr7f1h/wheels/c4/f9/e7/05e0371c5078725c26b6251a30b873f061a4387de03d3fcbcb
Successfully built dllogger
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import glob
import torch
import esm
import os
import re
from ctypes import sizeof
from collections import OrderedDict 
import sys
import shutil
import numpy as np
from tqdm import tqdm

# Pre-processing

## Set folder path of the PDB files

In [14]:
# Folder Path
path = r"/content/drive/MyDrive/PDB"

## Function to convert PDB to FASTA file

In [15]:
def convertPDBtoFASTA(pdb_file):
    aa3to1={
       'ALA':'A', 'VAL':'V', 'PHE':'F', 'PRO':'P', 'MET':'M',
       'ILE':'I', 'LEU':'L', 'ASP':'D', 'GLU':'E', 'LYS':'K',
       'ARG':'R', 'SER':'S', 'THR':'T', 'TYR':'Y', 'HIS':'H',
       'CYS':'C', 'ASN':'N', 'GLN':'Q', 'TRP':'W', 'GLY':'G',
       'MSE':'M', 'UNK':'X',
    }
    
    ca_pattern=re.compile("^ATOM\s{2,6}\d{1,5}\s{2}CA\s[\sA]([A-Z]{3})\s([\s\w])|^HETATM\s{0,4}\d   {1,5}\s{2}CA\s[\sA](MSE)\s([\s\w])")

    filename=os.path.basename(pdb_file).split('.')[0]
    dirname=os.path.dirname(pdb_file)
    chain_dict=dict()
    chain_list=[]

    fp=open(pdb_file,'rU')
    for line in fp.read().splitlines():
        if line.startswith("ENDMDL"):
            break
        match_list=ca_pattern.findall(line)
        if match_list:
            resn=match_list[0][0]+match_list[0][2]
            chain=match_list[0][1]+match_list[0][3]
            if chain in chain_dict:
                chain_dict[chain]+=aa3to1[resn]
            else:
                chain_dict[chain]=aa3to1[resn]
                chain_list.append(chain)
    fp.close()

    for chain in chain_list:

      if not os.path.exists(dirname+"/output/"+filename+"/"):
        os.makedirs(os.path.dirname(dirname+"/output/"+filename+"/"))

      fasta_file = open(dirname+"/output/"+filename+"/"+filename+".fasta", "a")
      fasta_file.write('>%s:%s\n%s\n'%(filename,chain,chain_dict[chain]))
      fasta_file.close()
      shutil.copy(dirname+"/"+filename+".pdb", dirname+"/output/"+filename+"/"+filename+".pdb")


## Converting all PDB files to FASTA

In [16]:
# Import Module
import os

# Change the directory
os.chdir(path)

# Read text File
def read_text_file(file_path):
	with open(file_path, 'r') as f:
		print(f.read())


# iterate through all file

print("Converting PDB files to FASTA files and writing to:"+ path + "/output" )
for file in tqdm(os.listdir()):
	# Check whether file is in text format or not
	if file.endswith(".pdb"):
		file_path = f"{path}/{file}"
		convertPDBtoFASTA(file_path)
	

print("\n")


Converting PDB files to FASTA files and writing to:/content/drive/MyDrive/PDB/output


  0%|          | 0/861 [00:00<?, ?it/s]<ipython-input-15-21d5b585e969>:17: DeprecationWarning: 'U' mode is deprecated
  fp=open(pdb_file,'rU')
100%|██████████| 861/861 [04:38<00:00,  3.09it/s]

## Function to get all second sequences in FASTA files

In [30]:
def get_second_line(file):
	with open(file, 'r') as f:
		return f.readlines()[1]


In [31]:
path = path+"/output"

# Change the directory
os.chdir(path)

# Read text File
def read_text_file(file_path):
	with open(file_path, 'r') as f:
		print(f.read())


# iterate through all file
ff=[]
all_folders = os.listdir()

print("Extracting the second sequence in all fasta files...")
for folder in tqdm(all_folders):
	os.chdir(path+"/"+folder)
	for file in os.listdir():
		if file.endswith(".fasta"):
			file_path = f"{path}/{folder}/{file}"
			ff.append((file,get_second_line(file_path)))


Extracting the second sequence in all fasta files...


100%|██████████| 430/430 [00:01<00:00, 419.14it/s]


In [32]:
ff

[('2a06_bundle.fasta',
  'WWNFGSLLGICLILQILTGLFLAIRYMHANGASMFFICLYMHVGRGLYTWNIGVILLLTVMATAFMGYFAFHFILPFIIMAIAMVHLIKDILGALLLILALMLLVLFALGGVLALAFSILILALIPLLSQCLFWALVADLLTLTWIGGITIGQLASVLYFLLILVLMPT\n'),
 ('2a65_bundle.fasta',
  'GLILAMAGNAVGLGNFLRFPMIPYIIAFLLVGIPLMWIEWAKILGVFGLWIPLVVAIYYVYIESWTLGFAIKFLVFAYIVFLITMFINVSILIRIERFAKIAMPTLFILAVFLVIRGVWIAAVGQIFFTLSLGFGAIITYLSGLTAATLNEKAEVILGGSILGFLWFFLLFFAGLTSSIAIMQPMIAFLAVLWTAAIVFFSAHLVMFLLDEMDFWAGTIGVVFFGLTELIIFFWIPRIYYYVMRYITPAFLAVLLVVWARWITRFYIIGLFLFLTFLVF\n'),
 ('2b2f_bundle.fasta',
  'ASTALVMLMVPGVGFFYAIALSFISLIITVLLWIFYGYSVSMYQMMFAAVTIAILTSAISSFILLSALWLTFVYAPFAHWLFAGGMVVHISSGFAALAVAMTLTLIGAALLWFGWFGFNGGSVVVTNTSAAVAGFVWMVIGWILGIVSGAIAGLAAITPAAKGAIVIGLVAGIVCYLAMDFRSLDAWAIHGIGGLWGSVAVGILLIAVASTTAYAFLVTLILAKAVDAAV\n'),
 ('3b4r_bundle.fasta',
  'HITFILFLVVIIGLSAVLFILLFVSVVLHELGAGPLVSFIIGIVLLIVSLYTLSLLNLMLGGFNLIPAFPMAANIGKSLALIMLLFGLIILILVSLFVYFGAE\n'),
 ('3b9w_bundle.fasta',
  'YSINILAMLLVGFGFLMVATTGTYLVVATGLPLYILLIYAEFAVATGLIAMGAVLFQYALLALFI

# ESM-2

## Load ESM-2 pretrained model

In [ ]:
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt


## *** For Google Colab: Limiting number of proteins to 10

Due to memory constraints

In [34]:
ff = ff[0:3]

ff

'2a06_bundle.fasta'

## Generate ESM-2 embeddings

In [ ]:
from math import e
print("Generating protein embeddings using ESM-2...")
for tuple in tqdm(ff):
  data = [tuple]
  batch_labels, batch_strs, batch_tokens = batch_converter(data)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      results = model(batch_tokens, repr_layers=[33], return_contacts=True)
  token_representations = results["representations"][33]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  
  # file = open(path+"/"+data[0][0].split(".")[0]+"/"+data[0][0].split(".")[0]+"_esm2.txt", "w+")
  if len(sequence_representations)< 2:
    # torch.save(sequence_representations[0], path+"/"+data[0][0].split(".")[0]+"/"+data[0][0].split(".")[0]+"_esm2.txt")
    np.savetxt(path+"/"+data[0][0].split(".")[0]+"/"+data[0][0].split(".")[0]+"_esm2.txt", torch.Tensor(sequence_representations[0]).numpy())
    # content = str(sequence_representations[0])
    # file.write(content)
    # file.close()
  else:
    print("more than one representations")
  # Saving the array in a text file
  # content = str(sequence_representations)
  # file.write(content)
  # file.close()


# # Look at the unsupervised self-attention map contact predictions
# import matplotlib.pyplot as plt
# for (_, seq), tokens_len, attention_contacts in zip(ff, batch_lens, results["contacts"]):
#     plt.matshow(attention_contacts[: tokens_len, : tokens_len])
#     plt.title(seq)
#     plt.show()

Generating protein embeddings using ESM-2...


100%|██████████| 3/3 [00:16<00:00,  5.61s/it]


In [ ]:
sequence_representations

[tensor([ 0.0435, -0.0686,  0.0389,  ...,  0.0573, -0.0584, -0.1091]),
 tensor([ 0.0514, -0.0141, -0.0357,  ...,  0.0535, -0.0585, -0.0356]),
 tensor([-0.0232, -0.0100,  0.0172,  ...,  0.0560, -0.0582, -0.0600])]

# AminoBert

## Download RGN2 and install dependencies

In [17]:
import os
import sys
import subprocess
from IPython import get_ipython
from IPython.utils import io

WORKDIR = './rgn2'
GIT_REPO = 'https://github.com/aqlaboratory/rgn2'
ENV_CONFIG = os.path.join(WORKDIR, 'environment.yml')
RGN2_PARAM_SOURCE_URL = 'https://huggingface.co/christinafl/rgn2'
RGN2_PARAMS_DIR = os.path.join(WORKDIR, 'resources')
RGN2_PARAM_RUN_DIR = os.path.join(RGN2_PARAMS_DIR, 'rgn2_runs')
RGN2_RUN_DIR = os.path.join(WORKDIR, 'runs')

AF2_GIT_REPO = 'https://github.com/deepmind/alphafold'
AF2_SOURCE_URL = 'https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar'
AF2_PARAMS_DIR = './alphafold/data/params'
AF2_PARAMS_PATH = os.path.join(AF2_PARAMS_DIR, os.path.basename(AF2_SOURCE_URL))

REFINER_DIR = os.path.join(WORKDIR, 'ter2pdb')
REFINER_PATH = os.path.join(REFINER_DIR, 'ModRefiner-l.zip')
REFINER_URL = 'https://zhanggroup.org/ModRefiner/ModRefiner-l.zip'

try:
  with io.capture_output() as captured:
    %cd '/content'

    # Different conda envs necessary due to conflicting dependencies.
    %shell rm -rf /opt/conda
    %shell wget -q -P /tmp \
      https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
        && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
        && rm /tmp/Miniconda3-latest-Linux-x86_64.sh
    
    PATH=%env PATH
    %env PATH=/opt/conda/bin:{PATH}

    CONDA_INIT = 'source /opt/conda/etc/profile.d/conda.sh && conda init'
    RGN2_ENV_INIT = f'{CONDA_INIT} && conda activate rgn2'

    # Download RGN2.
    %shell rm -rf {WORKDIR}
    %shell git clone {GIT_REPO} {WORKDIR}
    %shell {CONDA_INIT} && conda env create -f {ENV_CONFIG}

    # Download AF2 for AF2Rank-based refinement.
    AF2_ENV_INIT = f'{CONDA_INIT} && conda activate af2'

    %shell rm -rf alphafold
    %shell git clone --branch main {AF2_GIT_REPO} alphafold
    %shell {CONDA_INIT} && conda create -y -q --name af2 python=3.7
    %shell {AF2_ENV_INIT} && pip install -r ./alphafold/requirements.txt
    %shell {AF2_ENV_INIT} && pip install --no-dependencies ./alphafold
    %shell {AF2_ENV_INIT} && pip install --upgrade jax==0.3.17 \
      jaxlib==0.3.15+cuda11.cudnn805 \
      -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
    
    %shell mkdir --parents "{AF2_PARAMS_DIR}"
    %shell wget -O "{AF2_PARAMS_PATH}" "{AF2_SOURCE_URL}"
    %shell tar --extract --verbose --file="{AF2_PARAMS_PATH}" \
      --directory="{AF2_PARAMS_DIR}" --preserve-permissions
    %shell rm "{AF2_PARAMS_PATH}"

    # Download AminoBERT/RGN2 weights.
    %shell GIT_LFS_SKIP_SMUDGE=1 git clone "{RGN2_PARAM_SOURCE_URL}" "{RGN2_PARAMS_DIR}"
    %shell cd {RGN2_PARAMS_DIR} && git lfs pull
    %shell mv {RGN2_PARAM_RUN_DIR} {RGN2_RUN_DIR}

    # Download Modrefiner to initialize all atoms from CA trace.
    %shell wget -O {REFINER_PATH} {REFINER_URL}
    %shell unzip -o {REFINER_PATH} -d {REFINER_DIR}
    %shell rm {REFINER_PATH}

except subprocess.CalledProcessError:
  print(captured)
  raise

## Import python packages

In [18]:
%cd '/content/rgn2'

import os
import sys
import re
import hashlib
import json
import subprocess
from pathlib import Path
from IPython.utils import io
from google.colab import files

%reload_ext autoreload
%autoreload 2


sys.path.append('/content/alphafold')
from ter2pdb import ter2pdb

/content/rgn2


## Function to fold amino acid sequence 

In [42]:

def generateSeqParameters(sequence, jobname):
  # Remove whitespace
  sequence = "".join(sequence.split()).upper()
  jobname = "".join(jobname.split())

  jobname = re.sub(r'\W+', '', jobname)
  seq_hash = hashlib.blake2b(sequence.encode(), digest_size=3).hexdigest()
  seq_id = f'{jobname}_{seq_hash}'

  MAX_SEQUENCE_LENGTH = 1023

  # Remove all whitespaces, tabs and end lines; upper-case
  sequence = sequence.translate(str.maketrans('', '', ' \n\t')).upper()
  aatypes = set('ACDEFGHIKLMNPQRSTVWY')  # 20 standard aatypes
  if not set(sequence).issubset(aatypes):
    raise Exception(f'Input sequence contains non-amino acid letters: {set(sequence) - aatypes}. AlphaFold only supports 20 standard amino acids as inputs.')
  if len(sequence) > MAX_SEQUENCE_LENGTH:
    raise Exception(f'Input sequence is too long: {len(sequence)} amino acids, while the maximum is {MAX_SEQUENCE_LENGTH}. Please use the full AlphaFold system for long sequences.')

  run_inputs = {'sequence': sequence, 'seq_id': seq_id}
  with open("run_"+jobname+".json", "w") as f:
      json.dump(run_inputs, f)
      print(f)
  DATA_DIR = 'aminobert_output'
  RUN_DIR = 'runs/15106000'
# OUTPUT_DIR = 'output'
# REFINE_DIR = 'output/refine_model1'
# SEQ_PATH = os.path.join(DATA_DIR, f'{seq_id}.fa')
# TER_PATH = os.path.join(RUN_DIR, '1', 'outputsTesting', f'{seq_id}.tertiary')



In [43]:
for f in ff:
  print("Hello")
  generateSeqParameters(f[1], f[0])

Hello
<_io.TextIOWrapper name='run_2a06_bundlefasta.json' mode='w' encoding='UTF-8'>
Hello
<_io.TextIOWrapper name='run_2a65_bundlefasta.json' mode='w' encoding='UTF-8'>
Hello
<_io.TextIOWrapper name='run_2b2f_bundlefasta.json' mode='w' encoding='UTF-8'>


## Generate AminoBert Embeddings

In [28]:
%%bash
source /opt/conda/etc/profile.d/conda.sh && conda init
conda activate rgn2
python

import os
import sys
import json
sys.path.append(os.path.join(os.getcwd(), 'aminobert'))

import shutil
from aminobert.prediction import aminobert_predict_sequence
from data_processing.aminobert_postprocessing import aminobert_postprocess

DATA_DIR = 'aminobert_output'
DATASET_NAME = '1'
PREPEND_M = True
AMINOBERT_CHKPT_DIR = 'resources/aminobert_checkpoint/AminoBERT_runs_v2_uniparc_dataset_v2_5-1024_fresh_start_model.ckpt-1100000'

with open("run.json", "r") as f:
    run_inputs = json.load(f)

# Remove old data since AminoBERT combines entire directory to create dataset.
if os.path.exists(DATA_DIR):
  shutil.rmtree(DATA_DIR)
os.makedirs(DATA_DIR)

aminobert_predict_sequence(seq=run_inputs['sequence'], header=run_inputs['seq_id'],
                           prepend_m=PREPEND_M, checkpoint=AMINOBERT_CHKPT_DIR,
                           data_dir=DATA_DIR)
aminobert_postprocess(data_dir=DATA_DIR, dataset_name=DATASET_NAME, prepend_m=PREPEND_M)

no change     /opt/conda/condabin/conda
no change     /opt/conda/bin/conda
no change     /opt/conda/bin/conda-env
no change     /opt/conda/bin/activate
no change     /opt/conda/bin/deactivate
no change     /opt/conda/etc/profile.d/conda.sh
no change     /opt/conda/etc/fish/conf.d/conda.fish
no change     /opt/conda/shell/condabin/Conda.psm1
no change     /opt/conda/shell/condabin/conda-hook.ps1
no change     /opt/conda/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /opt/conda/etc/profile.d/conda.csh
no change     /root/.bashrc
No action taken.
Sequences being removed due to length: 0
Sequences being removed: [] []
Featurizing input
Writing numpy arrays





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2022-12-28 05:12:32.866843: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-12-28 05:12:32.873633: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-12-28 0